In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy



In [ ]:
df_rats = pd.read_csv('rat_counts.csv')
df_rats.head(2)

In [ ]:
## connect to mysql instance


engine = sqlalchemy.create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{3306}/{database}"
)

query = """
                select a.airbnb_id,
                    a.airbnb_name,
                    a.host_id,
                    h.host_name,
                    a.price,
                    a.room_type,
                    a.number_of_reviews,
                    a.availability_365,
                    r.number_of_rat_incidents,
                    n.neighbourhood_group,
                    n.neighbourhood,
                    l.latitude,
                    l.longitude
                from airbnb_ny.Apartments as a
                left join airbnb_ny.rats as r
                on a.airbnb_id = r.airbnb_id
                left join airbnb_ny.host as h
                on a.host_id = h.host_id
                left join airbnb_ny.location as l
                on a.airbnb_id = l.airbnb_id
                left join airbnb_ny.neighbourhoods as n
                on l.neighbourhood_id = n.combined_id;
                
                """

my_df = pd.read_sql(query,engine)


In [ ]:

df_price_rats = my_df[my_df['price'] < 6000]

neighborhoods = ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island']
colors = {
    'Manhattan': 'red',
    'Brooklyn': 'blue',
    'Queens': 'green',
    'Bronx': 'orange',
    'Staten Island': 'purple'
}

# Create subplots (1 row, 5 columns)
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(20, 4), sharey=True)

for ax, group in zip(axes, neighborhoods):
    data = df_price_rats[df_price_rats['neighbourhood_group'] == group]
    ax.scatter(
        data['number_of_rat_incidents'],
        data['price'],
        alpha=0.6,
        color=colors[group]
    )
    ax.set_title(group)
    ax.set_xlabel("Rat Incidents")
    if ax == axes[0]:
        ax.set_ylabel("Price")

plt.tight_layout()
plt.show()
